#### Imports

In [18]:
import datetime
import pandas as pd
import hopsworks
from functions import util
import os

In [19]:
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 

country="Sweden"
city = "Stockholm"
latitude = "59.3294"
longitude = "18.0687"

today = datetime.date.today()

2025-01-07 13:15:41,442 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 13:15:41,449 INFO: Initializing external client
2025-01-07 13:15:41,449 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 13:15:42,892 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207502


#### Get references to the Feature Groups </span>

In [20]:
# Retrieve feature groups
accidents_fg = fs.get_feature_group(
    name='accidents',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

#### Get Weather Forecast data</span>

In [21]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df['day_of_week'] = daily_df['date'].dt.dayofweek

daily_df

Coordinates 59.25°N 18.0°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,day_of_week
0,2025-01-07,3.20,0.0,24.251284,185.964386,Stockholm,1
1,2025-01-08,1.15,0.4,15.226764,173.211121,Stockholm,2
2,2025-01-09,-2.70,0.0,0.360000,270.000000,Stockholm,3
3,2025-01-10,-2.15,0.0,17.015474,353.927643,Stockholm,4
4,2025-01-11,-4.80,0.0,17.782686,328.240570,Stockholm,5
5,2025-01-12,-4.55,0.0,11.966953,276.911133,Stockholm,6
6,2025-01-13,4.05,0.2,22.033648,265.314178,Stockholm,0
7,2025-01-14,0.10,0.0,8.942214,49.899166,Stockholm,1
8,2025-01-15,2.30,0.0,14.764389,268.602844,Stockholm,2
9,2025-01-16,4.45,0.0,18.940241,278.746063,Stockholm,3


In [22]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
 6   day_of_week                  10 non-null     int32         
dtypes: datetime64[ns](1), float32(4), int32(1), object(1)
memory usage: 488.0+ bytes


In [23]:
weather_fg.insert(daily_df)

Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207502/jobs/named/weather_1_offline_fg_materialization/executions


(Job('weather_1_offline_fg_materialization', 'SPARK'), None)